In [7]:
import os, sys, platform, yaml, re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, FloatType, StringType
from pathlib import Path

# Preparación

In [2]:
conf_path = str(Path.cwd() / "config" / "ETL_config.yaml")
with open(conf_path, "r") as f:
    CFG = yaml.safe_load(f)

IS_WIN          = platform.system() == "Windows"
CSV_DIR         = CFG["paths"]["csv_base_dir"]["windows" if IS_WIN else "linux"]
PG_URL          = CFG["postgres"]["url"]
PG_USER         = CFG["postgres"]["user"]
PG_PASS         = CFG["postgres"]["pass"]
PG_SCHEMA       = CFG["postgres"]["schema_out"]["schema_name"]
PG_TABLE1       = CFG["postgres"]["schema_out"]["table1"]
PG_TABLE2       = CFG["postgres"]["schema_out"]["table2"]
PG_TABLE3       = CFG["postgres"]["schema_out"]["table3"]
FILES           = CFG["csv"]["files"]
NUM_PARTITIONS  = CFG["csv"]["num_partitions"]
JDBC_BATCHSIZE  = CFG["postgres"]["batchsize"]
JDBC_FETCHSIZE  = CFG["postgres"]["fetchsize"]
SPARK_LOCAL_DIR = CFG["spark"]["local_dirs"]["windows" if IS_WIN else "linux"]
NEO4J_URI  = CFG["neo4j"]["uri"]          # "bolt://localhost:7687"
NEO4J_USER = CFG["neo4j"]["user"]
NEO4J_PASS = CFG["neo4j"]["pass"]
NEO4J_DB   = CFG["neo4j"]["database"]

opts = {
    "url": NEO4J_URI,
    "authentication.type": "basic",
    "authentication.basic.username": NEO4J_USER,
    "authentication.basic.password": NEO4J_PASS,
    "database": NEO4J_DB,
}

os.environ["PYSPARK_PYTHON"] = sys.executable       # usa el Python del kernel actual
os.environ["JAVA_HOME"] = os.environ.get("JAVA_HOME", "/usr/lib/jvm/java-17-openjdk-amd64") #Chequear versión Windows después
Path(SPARK_LOCAL_DIR).mkdir(parents=True, exist_ok=True)


In [3]:
builder = (
    SparkSession.builder
    .master("local[*]")  
    .appName(CFG["spark"]["app_name"])
    .config("spark.sql.shuffle.partitions", str(CFG["spark"]["shuffle_partitions"]))
    .config("spark.driver.memory", CFG["spark"]["driver_memory"])
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.local.dir", SPARK_LOCAL_DIR)   
    #.config("spark.jars.packages", ",".join(CFG["spark"]["local_jars"]["linux"]))
    .config(
        "spark.jars.packages",
        ",".join([
            "org.postgresql:postgresql:42.7.4",
            "org.neo4j:neo4j-connector-apache-spark_2.12:5.3.10_for_spark_3"
        ])
    )
)

spark = builder.getOrCreate()
spark.sparkContext.setLogLevel("WARN")

jdbc_props = {
        "user": PG_USER,
        "password": PG_PASS,
        "driver": "org.postgresql.Driver",
        "fetchsize": str(JDBC_FETCHSIZE)
    }


25/11/09 17:39:21 WARN Utils: Your hostname, AsusMare resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface wlp2s0)
25/11/09 17:39:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.5.3-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/felpipe/.ivy2/cache
The jars for the packages stored in: /home/felpipe/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0ff4429a-5db4-4a70-9114-dce0f164e6c0;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.4 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in local-m2-cache
	found org.neo4j#caniuse-api;1.3.0 in local-m2-cache
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in local-m2-cache
	found org.jetbrains#annotations;13.0 in local-m2-cache
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in local-m2-cache
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in local-m2-cache
	

In [4]:
df_acc = (spark.read.format("jdbc")
        .option("url", PG_URL)
        .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE1}")
        .option("partitionColumn", "account")
        .option("lowerBound", 1)                  
        .option("upperBound", 2000000)
        .option("numPartitions", 16)               
        .options(**jdbc_props)
        .load())

df_txs = (spark.read.format("jdbc")
        .option("url", PG_URL)
        .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE2}")
        .option("partitionColumn", "id")
        .option("lowerBound", 1)
        .option("upperBound", 9500000)
        .option("numPartitions", 64)
        .options(**jdbc_props)
        .load())

acc_bounds = df_acc.select(
        F.min("account").cast("long").alias("lo"),
        F.max("account").cast("long").alias("hi")
    ).first()
acc_lo, acc_hi = int(acc_bounds["lo"]), int(acc_bounds["hi"])

df_sta =  (spark.read.format("jdbc")
        .option("url", PG_URL)
        .option("dbtable", f"{PG_SCHEMA}.{PG_TABLE3}")
        .option("partitionColumn", "account")       # particionamos por cuenta
        .option("lowerBound", acc_lo)
        .option("upperBound", acc_hi)
        .option("numPartitions", 64)                 # ajústalo a tu máquina/cluster
        .options(**jdbc_props)
        .load()
        .select(
            F.col("account").cast("long").alias("account"),
            F.col("date_time").alias("date_time"),
            F.col("txn_id").cast("long").alias("txn_id"),
            F.col("direction").alias("direction"),
            F.col("delta_amount").cast("double").alias("delta_amount"),
            F.col("running_balance").cast("double").alias("running_balance")
        )
    )

# Análisis univariado

## Setup inicial y overview

In [5]:
# Verificar conteos básicos
print("Accounts:", df_acc.count())
print("Transactions:", df_txs.count())
print("Statements:", df_sta.count())

# Visualizar schemas y sample
df_acc.printSchema()
df_txs.printSchema()
df_sta.printSchema()

df_acc.show(5)
df_txs.show(5)
df_sta.show(5)

Accounts: 855460


Transactions: 9504852


Statements: 19009704
root
 |-- account: long (nullable = true)
 |-- location: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- sender_account: long (nullable = true)
 |-- receiver_account: long (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_currency: string (nullable = true)
 |-- received_currency: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- is_laundering: integer (nullable = true)
 |-- laundering_type: string (nullable = true)

root
 |-- account: long (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- txn_id: long (nullable = true)
 |-- direction: string (nullable = true)
 |-- delta_amount: double (nullable = true)
 |-- running_balance: double (nullable = true)

+-------+--------+
|account|location|
+-------+--------+
|  99278|      UK|
| 114277|      UK|
|  28511|      UK|
|  23159|      UK|
|  58673|      UK|
+-------+--------+
only showing top 5 rows

+

## Chequeo de missing values y duplicados

In [8]:
def null_summary(df, name: str, include_empty_strings: bool = True, show_pct: bool = True):
    # Identifica tipos por columna
    numeric_nan_cols = {
        f.name for f in df.schema.fields
        if isinstance(f.dataType, (DoubleType, FloatType))
    }
    string_cols = {
        f.name for f in df.schema.fields
        if isinstance(f.dataType, StringType)
    }

    # Construye expresiones por columna
    exprs = []
    for c in df.columns:
        cond = F.col(c).isNull()
        if c in numeric_nan_cols:
            cond = cond | F.isnan(c)
        if include_empty_strings and c in string_cols:
            cond = cond | (F.trim(F.col(c)) == "")
        exprs.append(F.sum(F.when(cond, 1).otherwise(0)).alias(c))

    # Ejecuta el cómputo
    counts = df.select(exprs).first().asDict()
    total = df.count()

    print(f"Resumen de valores nulos en '{name}':")
    for c in df.columns:
        n = int(counts[c])
        if show_pct and total > 0:
            pct = 100.0 * n / total
            print(f'  - "{c}": {n:,} ({pct:.2f}%)')
        else:
            print(f'  - "{c}": {n:,}')
    

null_summary(df_acc, "accounts")
null_summary(df_txs, "transactions")
null_summary(df_sta, "statements")

Resumen de valores nulos en 'accounts':
  - "account": 0 (0.00%)
  - "location": 0 (0.00%)


Resumen de valores nulos en 'transactions':
  - "id": 0 (0.00%)
  - "date_time": 0 (0.00%)
  - "sender_account": 0 (0.00%)
  - "receiver_account": 0 (0.00%)
  - "amount": 0 (0.00%)
  - "payment_currency": 0 (0.00%)
  - "received_currency": 0 (0.00%)
  - "payment_type": 0 (0.00%)
  - "is_laundering": 0 (0.00%)
  - "laundering_type": 0 (0.00%)


[Stage 30:=================================================>      (57 + 7) / 64]

Resumen de valores nulos en 'statements':
  - "account": 0 (0.00%)
  - "date_time": 0 (0.00%)
  - "txn_id": 0 (0.00%)
  - "direction": 0 (0.00%)
  - "delta_amount": 0 (0.00%)
  - "running_balance": 0 (0.00%)


In [16]:
# Duplicados
print("Duplicados accounts:", df_acc.count() - df_acc.dropDuplicates().count())
print("Duplicados transactions:", df_txs.count() - df_txs.dropDuplicates().count())
print("Duplicados statements:", df_sta.count() - df_sta.dropDuplicates().count())

Duplicados accounts: 0


Duplicados transactions: 0


[Stage 44:==============================================>          (9 + 2) / 11]

Duplicados statements: 0


# Verificación de claves y consistencia básica

In [11]:
# Verificar que todas las cuentas usadas en transacciones existen en df_acc
unique_tx_accounts = (
    df_txs
    .select(F.col("sender_account").alias("account"))
    .union(df_txs.select(F.col("receiver_account").alias("account")))
    .distinct()
)

missing_accounts = unique_tx_accounts.join(df_acc, "account", "left_anti")
print("Cuentas en transacciones no presentes en accounts:", missing_accounts.count())


[Stage 37:>                                                         (0 + 8) / 9]

Cuentas en transacciones no presentes en accounts: 0


## Tipificación y valores anómalos

In [13]:
print(f'Montos negativos o cero (si no debieran existir): {df_txs.filter(F.col("amount") <= 0).count()}')

print('Fechas fuera de rango')
df_txs.agg(
    F.min("date_time").alias("min_date"),
    F.max("date_time").alias("max_date")
).show()

print(f'Verificar transacciones con misma cuenta de origen y destino: {df_txs.filter(F.col("sender_account") == F.col("receiver_account")).count()}')


Montos negativos o cero (si no debieran existir): 0
Fechas fuera de rango


+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2022-10-07 10:35:19|2023-08-23 10:57:12|
+-------------------+-------------------+



[Stage 57:=======================================================>(63 + 1) / 64]

Verificar transacciones con misma cuenta de origen y destino: 0


## Accounts

In [ ]:
df_acc

# Otros

In [6]:

df_nodes = (
    spark.read.format("org.neo4j.spark.DataSource")
    .options(**opts)
    .option("query", "MATCH (n) LIMIT 5 RETURN id(n) AS id, labels(n) AS labels")
    .load()
)
df_nodes.show(truncate=False)


+---+---------+
|id |labels   |
+---+---------+
|0  |[Account]|
|1  |[Account]|
|2  |[Account]|
|3  |[Account]|
|4  |[Account]|
+---+---------+



In [8]:
df_nodes = (
    spark.read.format("org.neo4j.spark.DataSource")
    .options(**opts)
    .option("query", "MATCH (n) -[r:TX]-> (m) LIMIT 1 RETURN n, r, m")
    .load()
)
df_nodes.show(truncate=False)

+----------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|n                                                                                             |r                                                                                                                                 